In [36]:
# Import the libraries required to estimate the index returns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Read in the training dataset provided by Optiver
all_data = pd.read_csv('train.csv')

In [ ]:
# For each date, calculate the stock returns and estimate the correlation matrix - use this correlation matrix in the PCA 
# to then estimate factor loadings of the first factor (likely the market factor) to each of the stocks
max_dates = max(all_data['date_id']) + 1
max_assets = max(all_data['stock_id']) + 1
daily_loadings_mkt = np.empty((max_dates, max_assets,))
daily_loadings_mkt[:] = np.nan

for date_id in range(max(all_data['date_id']) + 1):
    date_data = all_data[all_data['date_id'] == date_id]
    # Pivot the dataframe to create a wap time series for all the stocks
    date_ts_prices = date_data.pivot(index='time_id', columns='stock_id', values = 'wap')
    # Check for any empty columns and remove them
    date_ts_prices.dropna(how='all', axis=1, inplace=True)   
    stock_ids = sorted(date_ts_prices.columns.unique())
    num_assets = len(date_ts_prices.columns)
    date_ts_returns = date_ts_prices.pct_change(1)
    date_ts_returns = date_ts_returns.iloc[1:]
    corr_matrix = date_ts_returns.corr()
    pca = PCA(n_components=num_assets, svd_solver='full')
    pca.fit(corr_matrix)
    factor_loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
    factor_loadings_mkt = factor_loadings[:, 0]
    factor_loadings_mkt = factor_loadings_mkt / sum(factor_loadings_mkt)
    daily_loadings_mkt[date_id, stock_ids] = factor_loadings_mkt

np.savetxt('dailyfactorloadings.csv', daily_loadings_mkt, delimiter=',')